In [89]:
import json
import random
import textwrap
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from bellem.utils import jprint

In [ ]:
df = pd.read_json('../data/generated/reports/results.jsonl', lines=True)
print(len(df))
df.head()

2417


,id,n_hops,answer,hops,predicted_answer,reference_answers,exact_match,f1,fuzzy_match
0,2hop__460946_294723,2,Miquette Giraudy,[{'question': 'Who is the spouse of the Green ...,Miquette Giraudy,[Miquette Giraudy],1,1.0,1
1,2hop__252311_366220,2,Mike Medavoy,[{'question': 'Who founded the company that di...,Mike Medavoy,[Mike Medavoy],1,1.0,1
2,2hop__701895_752697,2,Nuevo Laredo Municipality,[{'question': 'What administrative territorial...,Nuevo Laredo Municipality,[Tamaulipas],0,0.0,0
3,2hop__259228_793698,2,German Aerospace Center,[{'question': 'Where is Ulrich Walter's employ...,German Aerospace Center,[Cologne],0,0.0,0
4,2hop__481349_302087,2,Bombardier Inc.,[{'question': 'Which company owns the manufact...,Bombardier Inc.,"[Bombardier, Bombardier Inc.]",1,1.0,1


In [91]:
def _parse_triplets(generation):
    seen_triplets_header = False
    for line in generation.split('\n'):
        if line.startswith('Triplets'):
            seen_triplets_header = True
            continue
        if line.startswith("Answer"):
            return
        if seen_triplets_header and line.strip():
            yield line.strip()

def parse_triplets(generation):
    return [triplet.strip() for triplet in _parse_triplets(generation) if triplet.strip()]

In [92]:
df['generation'] = df['hops'].map(lambda x: x[-1]['qa_result']['generation'])
df['triplets'] = df['generation'].map(parse_triplets)
df.head()

,id,n_hops,answer,hops,predicted_answer,reference_answers,exact_match,f1,fuzzy_match,generation,triplets
0,2hop__460946_294723,2,Miquette Giraudy,[{'question': 'Who is the spouse of the Green ...,Miquette Giraudy,[Miquette Giraudy],1,1.0,1,Triplets:\nMiquette Giraudy | partner | Steve ...,"[Miquette Giraudy | partner | Steve Hillage, S..."
1,2hop__252311_366220,2,Mike Medavoy,[{'question': 'Who founded the company that di...,Mike Medavoy,[Mike Medavoy],1,1.0,1,Triplets:\nMike Medavoy | co-founder of | Orio...,[Mike Medavoy | co-founder of | Orion Pictures...
2,2hop__701895_752697,2,Nuevo Laredo Municipality,[{'question': 'What administrative territorial...,Nuevo Laredo Municipality,[Tamaulipas],0,0.0,0,Triplets:\nCiudad Deportiva | located in | Nue...,"[Ciudad Deportiva | located in | Nuevo Laredo,..."
3,2hop__259228_793698,2,German Aerospace Center,[{'question': 'Where is Ulrich Walter's employ...,German Aerospace Center,[Cologne],0,0.0,0,Triplets:\nUlrich Walter | trained at | German...,[Ulrich Walter | trained at | German Aerospace...
4,2hop__481349_302087,2,Bombardier Inc.,[{'question': 'Which company owns the manufact...,Bombardier Inc.,"[Bombardier, Bombardier Inc.]",1,1.0,1,Triplets:\nLearjet 60 | manufactured by | Bomb...,[Learjet 60 | manufactured by | Bombardier Aer...


In [93]:
missing_triplets = ~(df['generation'].str.contains("Triplets"))
len(df.loc[missing_triplets])

0

In [94]:
success_mask = df['f1'] > 0.7
fail_df = df[~success_mask]
success_df = df[success_mask]
print(len(fail_df), len(success_df), len(df))
print("{:.3f}".format(len(success_df)/len(df)))

763 1654 2417
0.684


In [95]:
def format_paragraph(paragraph):
    return f"Paragraph {paragraph['idx']} - {paragraph['paragraph_text']}"

def present_hop(hop):
    print('-'*80)
    print(hop['context'])
    print('-'*80)
    print(hop['question'])
    print('-'*80)
    print(hop['qa_result']['generation'])

def present_row(row):
    print(row['id'])
    print()
    print("Success" if row['f1']>0.99 else "Fail")
    print()
    for hop in row['hops']:
        present_hop(hop)
    print()
    print(f"Reference: {row['reference_answers']}")

In [96]:
i = 0
row = df.iloc[i]
present_row(row)

2hop__460946_294723

Success

--------------------------------------------------------------------------------
# Miquette Giraudy
Miquette Giraudy (born 9 February 1953, Nice, France) is a keyboard player and vocalist, best known for her work in Gong and with her partner Steve Hillage. She and Hillage currently form the core of the ambient band System 7. In addition to her performances in music, she has also worked as an actress, film editor and writer. In each role, she has used different stage names.

# Green (Steve Hillage album)
Green is the fourth studio album by British progressive rock musician Steve Hillage. Written in spring 1977 at the same time as his previous album, the funk-inflected "Motivation Radio" (1977), "Green" was originally going to be released as "The Green Album" as a companion to "The Red Album" (the originally intended name for "Motivation Radio"). However, this plan was dropped and after a US tour in late 1977, "Green" was recorded alone, primarily in Dorking

In [97]:
i = 1
row = fail_df.iloc[i]
present_row(row)

2hop__259228_793698

Fail

--------------------------------------------------------------------------------
# Philae (spacecraft)
The lander is named after the Philae obelisk, which bears a bilingual inscription and was used along with the Rosetta Stone to decipher Egyptian hieroglyphs. "Philae" was monitored and operated from DLR's Lander Control Center in Cologne, Germany.

# Ulrich Walter
After two post-doc positions at the Argonne National Laboratory, Chicago, Illinois, and the University of California at Berkeley, California, he was selected in 1987 to join the German astronaut team. From 1988 to 1990, he completed basic training at the German Aerospace Center, and was then nominated to be in the prime crew for the second German Spacelab mission.
--------------------------------------------------------------------------------
Where is Ulrich Walter's employer headquartered?
--------------------------------------------------------------------------------
Triplets:
Ulrich Walter | t

In [98]:
df['n_triplets'] = df['triplets'].map(len)
df['n_triplets'].describe()

count    2417.000000
mean        2.894084
std         1.077480
min         1.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         9.000000
Name: n_triplets, dtype: float64

In [99]:
poor_triplets_mask = df['n_triplets'] < df['n_hops']
df[poor_triplets_mask].describe()

,n_hops,exact_match,f1,fuzzy_match,n_triplets
count,349.000000,349.000000,349.000000,349.000000,349.000000
mean,3.303725,0.547278,0.638062,0.587393,2.214900
std,0.746265,0.498474,0.440480,0.493010,0.736726
min,2.000000,0.000000,0.000000,0.000000,1.000000
25%,3.000000,0.000000,0.000000,0.000000,2.000000
50%,3.000000,1.000000,1.000000,1.000000,2.000000
75%,4.000000,1.000000,1.000000,1.000000,3.000000
max,4.000000,1.000000,1.000000,1.000000,3.000000


In [100]:
surprise_success_df = df[poor_triplets_mask&success_mask]
len(surprise_success_df)

199

In [102]:
i = random.randint(0, len(surprise_success_df))
row = surprise_success_df.iloc[i]
present_row(row)

3hop1__33845_42197_18397

Success

--------------------------------------------------------------------------------
# Korean War
On 27 June 1950, two days after the KPA invaded and three months before the Chinese entered the war, President Truman dispatched the United States Seventh Fleet to the Taiwan Strait, to prevent hostilities between the Nationalist Republic of China (Taiwan) and the People's Republic of China (PRC). On 4 August 1950, with the PRC invasion of Taiwan aborted, Mao Zedong reported to the Politburo that he would intervene in Korea when the People's Liberation Army's (PLA) Taiwan invasion force was reorganized into the PLA North East Frontier Force. China justified its entry into the war as a response to "American aggression in the guise of the UN".

# Imperialism
Trotsky, and others, believed that the revolution could only succeed in Russia as part of a world revolution. Lenin wrote extensively on the matter and famously declared that Imperialism was the highest sta